In [1]:
!pip install gradio pandas ollama pydantic_ai

In [2]:
!ollama --version

ollama version is 0.5.13


In [3]:
!ollama list

NAME               ID              SIZE      MODIFIED      
llama3:8b          365c0bd3c000    4.7 GB    9 minutes ago    
llama3.1:latest    46e0c10c039e    4.9 GB    18 hours ago     


In [4]:
!ollama pull llama3:8b

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest 
pulling 6a0746a1ec1a... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 4.7 GB                         
pulling 4fa551d4f938... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  12 KB                         
pulling 8ab4849b038c... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  254 B                         
pulling 577073ffcc6c... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  110 B                         
pulling 3f8eb4da87fa... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [5]:
import pydantic_ai.models
print(dir(pydantic_ai.models))

['ABC', 'ALLOW_MODEL_REQUESTS', 'AsyncIterator', 'Iterator', 'KnownModelName', 'Literal', 'Model', 'ModelMessage', 'ModelRequestParameters', 'ModelResponse', 'ModelResponsePartsManager', 'ModelResponseStreamEvent', 'ModelSettings', 'StreamedResponse', 'TYPE_CHECKING', 'Usage', 'UserError', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_annotations', '_cached_async_http_client', 'abstractmethod', 'asynccontextmanager', 'cache', 'cached_async_http_client', 'check_allow_model_requests', 'contextmanager', 'dataclass', 'datetime', 'field', 'get_user_agent', 'httpx', 'infer_model', 'instrumented', 'override_allow_model_requests', 'wrapper']


In [6]:
import pandas as pd
import gradio as gr
import ollama
import matplotlib.pyplot as plt
import seaborn as sns
from pydantic_ai.models import Model

In [7]:
MODEL_NAME = "llama3:8b"  

df = None  # Global variable to store the uploaded DataFrame


class CSVQueryModel(Model):
    """AI model processes natural language queries about CSV data."""
    question: str

# Function to load CSV with validation
def load_csv(file):
    global df
    try:
        if not file:
            return "⚠️ No file uploaded!", gr.update(choices=[]), gr.update(choices=[])
        
        df = pd.read_csv(file.name)
        
        if df.empty:
            return "⚠️ Uploaded CSV is empty!", gr.update(choices=[]), gr.update(choices=[])
        
        column_list = df.columns.tolist()
        return (
            f"✅ CSV Loaded Successfully! Columns: {', '.join(column_list)}",
            gr.update(choices=column_list, value=column_list[0] if column_list else None),
            gr.update(choices=column_list, value=column_list[1] if len(column_list) > 1 else None)
        )
    except Exception as e:
        return (f"❌ Error loading CSV: {str(e)}", gr.update(choices=[]), gr.update(choices=[]))

# Function to process query using Llama3:8b model

def answer_query(question):
    global df
    if df is None:
        return "⚠️ Please upload a CSV file first."
    
    try:
        csv_data = df.head(50).to_json(orient="records")  # Convert sample of CSV data to JSON format

        # Prepare LLM prompt
        prompt = f"""
        You are an AI assistant answering questions about CSV data.
        Given the CSV sample: {csv_data}
        Answer concisely: {question}
        """
        
        # Query the Llama model
        response = ollama.chat(model=MODEL_NAME, messages=[{"role": "user", "content": prompt}])
        
        return response["message"]["content"]  # ✅ Ensure correct response extraction
    
    except Exception as e:
        return f"❌ Error processing query: {str(e)}"


# Function to plot different graphs
def plot_graph(x_axis, y_axis, graph_type):
    global df
    if df is None:
        return "⚠️ Please upload a CSV file first."
    
    if x_axis not in df.columns or y_axis not in df.columns:
        return "⚠️ Invalid column selection."
    
    plt.figure(figsize=(8, 5))
    
    try:
        if graph_type == "Scatter Plot":
            sns.scatterplot(data=df, x=x_axis, y=y_axis)
        elif graph_type == "Line Chart":
            sns.lineplot(data=df, x=x_axis, y=y_axis)
        elif graph_type == "Bar Chart":
            sns.barplot(data=df, x=x_axis, y=y_axis)
        elif graph_type == "Histogram":
            sns.histplot(df[x_axis], bins=20, kde=True)
        
        plt.xlabel(x_axis)
        plt.ylabel(y_axis)
        plt.title(f"{graph_type}: {x_axis} vs {y_axis}")
        
        # Save plot for display in Gradio
        plot_path = "plot.png"
        plt.savefig(plot_path)
        plt.close()
        return plot_path
    except Exception as e:
        return f"❌ Error generating graph: {str(e)}"

# Gradio UI
with gr.Blocks() as app:
    gr.Markdown("# 🦙 AI-powered CSV Analysis with Llama 3 & Pydantic AI 🦙")

    file_input = gr.File(label="Upload CSV", type="filepath")
    upload_button = gr.Button("Load CSV")
    status_output = gr.Textbox(label="Status", interactive=False)

    question_input = gr.Textbox(label="Ask a question about the CSV")
    submit_button = gr.Button("Submit")
    response_output = gr.Textbox(label="LLM Response", interactive=False)

    # Dynamic dropdown for column selection
    x_axis_dropdown = gr.Dropdown(label="X-axis", choices=[], interactive=True)
    y_axis_dropdown = gr.Dropdown(label="Y-axis", choices=[], interactive=True)
    graph_type_dropdown = gr.Radio(
        choices=["Scatter Plot", "Line Chart", "Bar Chart", "Histogram"],
        label="Graph Type"
    )

    plot_button = gr.Button("Generate Graph")
    graph_output = gr.Image()

    # Button actions
    upload_button.click(load_csv, inputs=[file_input], outputs=[status_output, x_axis_dropdown, y_axis_dropdown])
    submit_button.click(answer_query, inputs=[question_input], outputs=[response_output])
    plot_button.click(plot_graph, inputs=[x_axis_dropdown, y_axis_dropdown, graph_type_dropdown], outputs=[graph_output])

# Launch the app
app.launch()

* Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.
